### Test Docling

In [ ]:
from docling.document_converter import DocumentConverter

file = '../../data/papers/Rajkumar et al. - 2022 - Evaluating the Text-to-SQL Capabilities of Large L.pdf'
converter = DocumentConverter()
document = converter.convert(file)

print(document.document.origin.filename)

c:\Users\Arm\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\cpp_extension.py:382: UserWarning: Error checking compiler version for cl: [WinError 2] The system cannot find the file specified
  warnings.warn(f'Error checking compiler version for {compiler}: {error}')
c:\Users\Arm\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\cpp_extension.py:1964: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(
Could not load the custom kernel for multi-scale deformable attention: Command '['where', 'cl']' returned non-zero exit status 1.
Could not load the custom kernel for multi-scale deformable attention: DLL load failed while importing MultiScaleDeformableAttention: The specified module could not be found.
Could not load the custom kernel for multi-scale deformable attention: DLL load failed while importing Mult

Rajkumar et al. - 2022 - Evaluating the Text-to-SQL Capabilities of Large L.pdf


### Test Chunking

In [4]:
from docling.chunking import HybridChunker
from docling_core.transforms.chunker.tokenizer.huggingface import HuggingFaceTokenizer
from transformers import AutoTokenizer

TOKENIZER_MODEL="BAAI/bge-m3"
MAX_TOKENS=8192

tokenizer = HuggingFaceTokenizer(
        tokenizer=AutoTokenizer.from_pretrained(TOKENIZER_MODEL),
        max_tokens=MAX_TOKENS,
    )

chunker = HybridChunker(
    tokenizer=tokenizer,
    merge_peers=True
)

chunk_iter = chunker.chunk(dl_doc=document.document)
chunks = list(chunk_iter)

In [ ]:
# Checking the documents metadata
chunks[0].model_dump()['meta']

{'schema_name': 'docling_core.transforms.chunker.DocMeta',
 'version': '1.0.0',
 'doc_items': [{'self_ref': '#/texts/2',
   'parent': {'cref': '#/body'},
   'children': [],
   'content_layer': <ContentLayer.BODY: 'body'>,
   'label': <DocItemLabel.TEXT: 'text'>,
   'prov': [{'page_no': 1,
     'bbox': {'l': 142.048,
      't': 724.8820146484375,
      'r': 455.715,
      'b': 712.6010146484375,
      'coord_origin': <CoordOrigin.BOTTOMLEFT: 'BOTTOMLEFT'>},
     'charspan': (0, 61)}]},
  {'self_ref': '#/texts/3',
   'parent': {'cref': '#/body'},
   'children': [],
   'content_layer': <ContentLayer.BODY: 'body'>,
   'label': <DocItemLabel.TEXT: 'text'>,
   'prov': [{'page_no': 1,
     'bbox': {'l': 72.983,
      't': 710.9350146484375,
      'r': 525.283,
      'b': 698.6730146484375,
      'coord_origin': <CoordOrigin.BOTTOMLEFT: 'BOTTOMLEFT'>},
     'charspan': (0, 93)},
    {'page_no': 1,
     'bbox': {'l': 138.186,
      't': 693.0410146484376,
      'r': 460.079,
      'b': 672.0820

In [12]:
processed_chunks = [
    {
        "text": chunk.text,
        "metadata": {
            "filename": chunk.meta.origin.filename,
            "page_numbers": [
                page_no
                for page_no in sorted(
                    set(
                        prov.page_no
                        for item in chunk.meta.doc_items 
                        for prov in item.prov
                    )
                )
            ] or None,
            "title": chunk.meta.headings[0] if chunk.meta.headings else None
        },
    }
    for chunk in chunks
]

In [13]:
processed_chunks

[{'text': 'Nitarshan Rajkumar 1 ∗ , Raymond Li 2 , Dzmitry Bahdanau 2345\n1 University of Cambridge, 2 ServiceNow, 3 Mila, 4 McGill University, 5 Canada CIFAR AI Chair nr500@cam.ac.uk , {raymond.li,dzmitry.bahdanau}@servicenow.com https://github.com/nitarshan/codex-text2sql',
  'metadata': {'filename': 'Rajkumar et al. - 2022 - Evaluating the Text-to-SQL Capabilities of Large L.pdf',
   'page_numbers': [1],
   'title': 'Evaluating the Text-to-SQL Capabilities of Large Language Models'}},
 {'text': 'Weperform an empirical evaluation of Text-toSQLcapabilities of the Codex language model. We find that, without any finetuning , Codex is a strong baseline on the Spider benchmark; we also analyze the failure modes of Codex in this setting. Furthermore, we demonstrate on the GeoQuery and Scholar benchmarks that a small number of in-domain examples provided in the prompt enables Codex to perform better than state-of-the-art models finetuned on such few-shot examples.\nFinetuned, VA = . Finetun

### Test LanceDB

In [ ]:
import lancedb
from lancedb.embeddings import get_registry
from lancedb.pydantic import LanceModel, Vector
from typing import List
db = lancedb.connect("data/lancedb")

func = get_registry().get("huggingface").create(name="BAAI/bge-m3")

class ChunkMetadata(LanceModel):
    filename: str | None
    page_numbers: List[int] | None
    title: str | None

class Chunks(LanceModel):
    text: str = func.SourceField()
    vector: Vector(func.ndims()) = func.VectorField()
    metadata: ChunkMetadata

table = db.create_table("docling", schema=Chunks, mode="overwrite")

In [16]:
table.add(processed_chunks)

ValueError: Field 'title' not found in target schema